<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Raw-Data" data-toc-modified-id="Raw-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Raw Data</a></span></li><li><span><a href="#Construct-Trade-Weight-Matrix" data-toc-modified-id="Construct-Trade-Weight-Matrix-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Construct Trade Weight Matrix</a></span></li></ul></li></ul></div>

# Initialization

In [1]:
import pandas as pd, numpy as np, dask.dataframe as ddf, seaborn as sns, pickle, pyflux as pf
import quandl
import sys, os, socket, string
from dask import delayed
from itertools import permutations, product
from functools import reduce
import pandas_datareader as pdr
import xlrd

scriptDir = os.path.dirname(os.path.realpath(sys.argv[0]))   # get script directory
try: 
    genDir = "C:/Users/benmo/OneDrive/GitHub/DataPlayground" #get general functions path
    os.chdir(genDir)
except: 
    genDir = "D:/benmo/OneDrive/GitHub/DataPlayground"
    os.chdir(genDir)

os.chdir(genDir)

from General import * # import custom 'general' functions
from DataEther import * # import custom 'data' functions

os.chdir(scriptDir)

In [2]:
from sympy.interactive.printing import init_printing
init_printing(use_unicode=False, wrap_line=True, no_global=True)
from sympy import *

In [3]:
%load_ext rpy2.ipython

In [4]:
fpath = MyComp.financePath  #path to financial econ folder

cntryBook = xlrd.open_workbook(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx")
sheets = cntryBook.sheet_names()

data = pd.DataFrame([])
idx = pd.IndexSlice

# Data

## Raw Data

In [5]:
for sheet in sheets:
    temp = pd.read_excel(fpath + "/data/gvar/Country Data (1979Q1-2011Q2).xlsx", sheet_name=sheet)
    temp['Country'] = sheet
    temp.set_index([temp['Country'], temp['date']], inplace=True)
    data = data.append(temp)
    
data = data.sort_index()

tradeMatrix = pd.read_excel(fpath + "/data/gvar/Trade Matrices (1980-2010).xls")
cntryCodes = pd.read_excel(fpath + "/data/gvar/Country Codes.xls", index_col='Country')

## Construct Trade Weight Matrix 

In [ ]:
tradeMatrix['country_primary'] = tradeMatrix['country_primary'].apply(lambda x, code=cntryCodes: 
                                     code[code.values == x].index[0])

countries = tradeMatrix[tradeMatrix.year == 1980]['country_primary'].tolist()
tradeMatrix.rename(columns=lambda x, names=cntryCodes: 
                   names[names.values == int(x[1:])].index[0] if [x[1:]] in names
                   .values.astype(str).tolist() else x, 
                   inplace=True)



W = tradeMatrix.set_index([tradeMatrix.country_primary, 
                           pd.to_datetime(tradeMatrix.astype(str).year)])

## fgbfdhdf